<a href="https://colab.research.google.com/github/vkjadon/State-Estimation/blob/main/4-kalmanFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kalman filter: quick recap

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

Kalman filter (KF) is one of the most used state estimation algorithms in robotics applications. We may already know the motion model of a robot that allows us to know how the state of the robot evolves over time given its initial state. However, we always have to deal with imperfect motion model for various reasons (e.g., imperfect parts, etc.). Although robots typically have various sensors to also help in estimating the state of the robot, sensors are also noisy. With Kalman filter, we can combine the prediction from our motion model with the sensor measurement to get a better state estimate.

Remember the assumptions used in Kalman filter:
1. Linear dynamics with Gaussian noise $\mathbf{x}_{t} = A \mathbf{x}_{t-1} + B \mathbf{u}_{t-1} + \mathbf{w}_{t}$ where $\mathbf{w}_{t} \sim \mathcal{N}(0, Q)$
2. Linear measurement model with Gaussian noise $\mathbf{z}_t = H \mathbf{x}_t + \mathbf{n}_t$ where $\mathbf{n}_t \sim \mathcal{N}(0, R)$
3. Gaussian prior $bel(\mathbf{x}_0) \sim \mathcal{N}(\mu_0, \Sigma_0)$

In Kalman filter, there are two main steps that we need to perform. These are the prediction and update steps. We perform the prediction step to compute our new belief state after we apply a control signal $u_t$ to our robot following the motion model. After we apply a control signal $u_t$, we assume to have received a sensor measurement $z_t$, and we perform the update step to update the belief state based on the sensor measurement.

Prediction step:
1. $\mu_{t|t-1} = A\mu_{t-1|t-1} + B\mathbf{u}_{t-1}$
2. $\Sigma_{t|t-1} = A\Sigma_{t-1|t-1}A^T + Q$

Update step:
1. Get measurement $\mathbf{z}_{t}$
2. Compute mean and covariance of the prediction residual $\delta_{\mu} = \mathbf{z}_{t} - H\mu_{t|t-1}$ and $\delta_{\Sigma} = H \Sigma_{t|t-1} H^T + R$
3. Compute Kalman gain $K_{t} = \Sigma_{t|t-1} H^T \delta_{\Sigma}^{-1}$
4. Compute mean and covariance of the belief $\mu_{t | t} = \mu_{t | t-1} + K\delta_{\mu}$ and $\Sigma_{t | t} = \Sigma_{t|t-1} - KH\Sigma_{t|t-1}$
